<a href="https://colab.research.google.com/github/abdipourasl/Machine-Learning-1402/blob/main/ML_MidTerm2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="alert alert-block alert-success">
<h1>Machine Learning MidTermProject<h1>
Amin Abdipour 401133011</h1>
</div>

# Add library & Clone

In [ ]:
!git clone https://github.com/jakesnell/ove-polya-gamma-gp.git


Cloning into 'ove-polya-gamma-gp'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 57 (delta 10), reused 53 (delta 8), pack-reused 0
Receiving objects: 100% (57/57), 53.70 KiB | 1.63 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [ ]:
pip install imagecorruptions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.6 MB/s eta 0:00:00


In [ ]:
pip install gpytorch numpy pypolyagamma sacred tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.6/175.6 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00
  Created wheel for pypolyagamma: filename=pypolyagamma-1.2.3-cp310-cp310-linux_x86_64.whl size=778506 sha256=24a9f7398dfa62c52191468fd0a242367d46c03c2f6933c98f73a2982b914c37
  Stored in directory: /root/.cache/pip/wheels/56/83/bd/0378099f2c88026b81d825c119a47b94f100f1e748b2be04fa
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-n

In [ ]:
pip install pypolyagamma

# Add Dataset

In [ ]:
#!/usr/bin/env bash
!wget http://www.vision.caltech.edu/visipedia-data/CUB-200-2011/CUB_200_2011.tgz
!tar -zxvf CUB_200_2011.tgz
!python /content/ove-polya-gamma-gp/filelists/CUB/write_CUB_filelist.py

--2024-02-22 02:18:38--  http://www.vision.caltech.edu/visipedia-data/CUB-200-2011/CUB_200_2011.tgz
Resolving www.vision.caltech.edu (www.vision.caltech.edu)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to www.vision.caltech.edu (www.vision.caltech.edu)|185.199.108.153|:80... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-02-22 02:18:38 ERROR 404: Not Found.

tar (child): CUB_200_2011.tgz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
Traceback (most recent call last):
  File "/content/ove-polya-gamma-gp/filelists/CUB/write_CUB_filelist.py", line 16, in <module>
    folder_list = [f for f in listdir(data_path) if isdir(join(data_path, f))]
FileNotFoundError: [Errno 2] No such file or directory: '/content/CUB_200_2011/images'


In [ ]:
#!/usr/bin/env bash
!wget https://github.com/NanqingD/DAOSL/raw/master/data/emnist.zip
!unzip emnist.zip
!python invert_emnist.py
!python write_cross_char_valnovel_filelist.py

Streaming output truncated to the last 5000 lines.
 extracting: emnist/42/117.png       
 extracting: emnist/42/118.png       
 extracting: emnist/42/119.png       
 extracting: emnist/42/12.png        
 extracting: emnist/42/120.png       
 extracting: emnist/42/121.png       
 extracting: emnist/42/122.png       
 extracting: emnist/42/123.png       
 extracting: emnist/42/124.png       
 extracting: emnist/42/125.png       
 extracting: emnist/42/126.png       
 extracting: emnist/42/127.png       
 extracting: emnist/42/128.png       
 extracting: emnist/42/129.png       
 extracting: emnist/42/13.png        
 extracting: emnist/42/130.png       
 extracting: emnist/42/131.png       
 extracting: emnist/42/132.png       
 extracting: emnist/42/133.png       
 extracting: emnist/42/134.png       
 extracting: emnist/42/135.png       
 extracting: emnist/42/136.png       
 extracting: emnist/42/137.png       
 extracting: emnist/42/138.png       
 extracting: emnist/42/139.png       

# Train Loop

In [ ]:
import numpy as np
import torch
import random
import torch.optim
import os

import configs
import backbone
from data.datamgr import SimpleDataManager, SetDataManager
from methods.baselinetrain import BaselineTrain
from methods.ove_polya_gamma_gp import OVEPolyaGammaGP, PredictiveOVEPolyaGammaGP
from methods.logistic_softmax_gp import LogisticSoftmaxGP, PredictiveLogisticSoftmaxGP
from methods.bayesian_maml import BayesianMAML, ChaserBayesianMAML
from methods.gpnet import GPNet
from methods.protonet import ProtoNet
from methods.matchingnet import MatchingNet
from methods.relationnet import RelationNet
from methods.maml import MAML
from io_utils import model_dict

from methods.ove_polya_gamma_gp import kernel_ingredient

from tensorboardX import SummaryWriter

from sacred import Experiment
from sacred.observers import FileStorageObserver


EXPERIMENT_NAME = "train"
RUN_DIR = "runs"


def get_save_dir():
    return os.path.join("runs", EXPERIMENT_NAME)

ex = Experiment(EXPERIMENT_NAME, ingredients=[kernel_ingredient])
ex.observers.append(FileStorageObserver(get_save_dir()))


@ex.config
def get_config():
    # Seed for Numpy and pyTorch. Default: 0 (None)
    seed = 0

    # CUB/miniImagenet/cross/omniglot/cross_char
    dataset = "CUB"

    # model: Conv{4|6} / ResNet{10|18|34|50|101}
    model = "Conv4"

    # relationnet_softmax replace L2 norm with softmax to expedite training,
    # maml_approx use first-order approximation in the gradient for efficiency
    # ove_polya_gamma_gp/predictive_ove_polya_gamma_gp/baseline/baseline++/protonet/matchingnet/relationnet{_softmax}/maml{_approx}
    method = "baseline"

    # baseline and baseline++ would ignore this parameter
    # class num to classify for training
    train_n_way = 5

    # baseline and baseline++ only use this parameter in finetuning
    # class num to classify for testing (validation)
    test_n_way = 5

    # baseline and baseline++ only use this parameter in finetuning
    # number of labeled data in each class, same as n_support
    n_shot = 5

    # still required for save_features.py and test.py to find the model path correctly
    # perform data augmentation or not during training
    train_aug = False

    # make it larger than the maximum label value in base class
    # total number of classes in softmax, only used in baseline
    num_classes = 200

    # Save frequency
    save_freq = 10

    # Starting epoch
    start_epoch = 0

    # for meta-learning methods, each epoch contains 100 episodes.
    # The default epoch number is dataset dependent. See train.py
    # Stopping epoch
    stop_epoch = -1

    # optimizer to use
    optimization = "Adam"

    # num_draws for ove_polya_gamma_gp
    num_draws = None

    # num_steps for ove_polya_gamma_gp
    num_steps = None

    sigma = None

    # tag (for logging purposes)
    tag = "default"


@ex.capture
def get_base_file(dataset):
    if dataset == "cross":
        return configs.data_dir["miniImagenet"] + "all.json"
    elif dataset == "cross_char":
        return configs.data_dir["omniglot"] + "noLatin.json"
    else:
        return configs.data_dir[dataset] + "base.json"


@ex.capture
def get_val_file(dataset):
    if dataset == "cross":
        return configs.data_dir["CUB"] + "val.json"
    elif dataset == "cross_char":
        return configs.data_dir["emnist"] + "val.json"
    else:
        return configs.data_dir[dataset] + "val.json"


@ex.capture
def get_image_size(model, dataset):
    if "Conv" in model:
        if dataset in ["omniglot", "cross_char"]:
            return 28
        else:
            return 84
    else:
        return 224


@ex.capture
def get_stop_epoch(n_shot, method, dataset, stop_epoch):
    if stop_epoch == -1:
        if method in ["baseline", "baseline++"]:
            if dataset in ["omniglot", "cross_char"]:
                stop_epoch = 5
            elif dataset in ["CUB"]:
                # This is different as stated in the open-review paper. However,
                # using 400 epoch in baseline actually lead to over-fitting
                stop_epoch = 200
            elif dataset in ["miniImagenet", "cross"]:
                stop_epoch = 400
            else:
                stop_epoch = 400  # default
        else:  # meta-learning methods
            if n_shot == 1:
                stop_epoch = 600
            elif n_shot == 5:
                stop_epoch = 400
            else:
                stop_epoch = 600  # default

    return stop_epoch


@ex.capture
def get_n_query(test_n_way, train_n_way):
    # if test_n_way is smaller than train_n_way, reduce n_query to keep batch size small
    return max(1, int(16 * test_n_way / train_n_way))


@ex.capture
def get_base_loader(method, train_n_way, n_shot, train_aug):
    if method in ["baseline", "baseline++"]:
        base_datamgr = SimpleDataManager(get_image_size(), batch_size=16)
    else:
        base_datamgr = SetDataManager(
            get_image_size(), n_query=get_n_query(), n_way=train_n_way, n_support=n_shot
        )  # n_eposide=100

    return base_datamgr.get_data_loader(get_base_file(), aug=train_aug)


@ex.capture
def get_val_loader(method, test_n_way, n_shot, train_aug):
    if method in ["baseline", "baseline++"]:
        val_datamgr = SimpleDataManager(get_image_size(), batch_size=64)
    else:
        val_datamgr = SetDataManager(
            get_image_size(), n_query=get_n_query(), n_way=test_n_way, n_support=n_shot
        )

    return val_datamgr.get_data_loader(get_val_file(), aug=False)


@ex.capture
def validate_config(dataset, model, method, num_classes, train_aug):
    # dataset checks
    if dataset in ["omniglot", "cross_char"]:
        assert (
            model == "Conv4S" and train_aug is False
        ), "omniglot only support Conv4 without augmentation"

    # method checks
    if method in ["baseline", "baseline++"]:
        if dataset == "omniglot":
            assert (
                num_classes >= 4112
            ), "class number need to be larger than max label id in base class"
        if dataset == "cross_char":
            assert (
                num_classes >= 1597
            ), "class number need to be larger than max label id in base class"


@ex.capture
def get_model(
    model,
    dataset,
    method,
    num_classes,
    train_n_way,
    n_shot,
    num_draws,
    num_steps,
    sigma,
):
    train_few_shot_params = dict(n_way=train_n_way, n_support=n_shot)

    if method == "baseline":
        return BaselineTrain(model_dict[model], num_classes)
    elif method == "baseline++":
        return BaselineTrain(model_dict[model], num_classes, loss_type="dist")
    elif method == "ove_polya_gamma_gp":
        model = OVEPolyaGammaGP(model_dict[model], **train_few_shot_params)
        if num_draws is not None:
            model.num_draws = num_draws
        if num_steps is not None:
            model.num_steps = num_steps
        if sigma is not None:
            model.kernel.sigma = sigma
        return model
    elif method == "predictive_ove_polya_gamma_gp":
        model = PredictiveOVEPolyaGammaGP(
            model_dict[model], **train_few_shot_params, fast_inference=True
        )
        if num_draws is not None:
            model.num_draws = num_draws
        if num_steps is not None:
            model.num_steps = num_steps
        if sigma is not None:
            model.kernel.sigma = sigma
        return model
    elif method == "logistic_softmax_gp":
        model = LogisticSoftmaxGP(model_dict[model], **train_few_shot_params)
        if num_draws is not None:
            model.num_draws = num_draws
        if num_steps is not None:
            model.num_steps = num_steps
        if sigma is not None:
            model.kernel.sigma = sigma
        return model
    elif method == "predictive_logistic_softmax_gp":
        model = PredictiveLogisticSoftmaxGP(model_dict[model], **train_few_shot_params)
        if num_draws is not None:
            model.num_draws = num_draws
        if num_steps is not None:
            model.num_steps = num_steps
        if sigma is not None:
            model.kernel.sigma = sigma
        return model
    elif method == "bayesian_maml":
        model = BayesianMAML(
            model_dict[model],
            **train_few_shot_params,
            num_draws=num_draws,
            num_steps=num_steps
        )
        return model
    elif method == "chaser_bayesian_maml":
        return ChaserBayesianMAML(
            model_dict[model],
            **train_few_shot_params,
            num_draws=num_draws,
            num_steps=num_steps
        )
    elif method == "gpnet":
        model = GPNet(model_dict[model], **train_few_shot_params)
        model.init_summary()
        return model
    elif method == "protonet":
        return ProtoNet(model_dict[model], **train_few_shot_params)
    elif method == "matchingnet":
        return MatchingNet(model_dict[model], **train_few_shot_params)
    elif method in ["relationnet", "relationnet_softmax"]:
        if model == "Conv4":
            feature_model = backbone.Conv4NP
        elif model == "Conv6":
            feature_model = backbone.Conv6NP
        elif model == "Conv4S":
            feature_model = backbone.Conv4SNP
        else:
            feature_model = lambda: model_dict[model](flatten=False)
        loss_type = "mse" if method == "relationnet" else "softmax"
        return RelationNet(feature_model, loss_type=loss_type, **train_few_shot_params)
    elif method in ["maml", "maml_approx"]:
        backbone.ConvBlock.maml = True
        backbone.SimpleBlock.maml = True
        backbone.BottleneckBlock.maml = True
        backbone.ResNet.maml = True
        model = MAML(
            model_dict[model], approx=(method == "maml_approx"), **train_few_shot_params
        )
        if dataset in [
            "omniglot",
            "cross_char",
        ]:  # maml use different parameter in omniglot
            model.n_task = 32
            model.task_update_num = 1
            model.train_lr = 0.1
        return model
    else:
        raise ValueError("unknown method {}".format(method))


def _set_seed(seed, verbose=True):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        if verbose:
            print("[INFO] Setting SEED: " + str(seed))
    else:
        if verbose:
            print("[INFO] Setting SEED: None")


def train(
    base_loader,
    val_loader,
    model,
    optimizer,
    start_epoch,
    stop_epoch,
    checkpoint_dir,
    writer,
    save_freq,
    max_acc,
    _run,
):
    print("Total epochs: {:d}".format(stop_epoch))

    for epoch in range(start_epoch, stop_epoch):
        model.train()
        train_loss = model.train_loop(epoch, base_loader, optimizer)
        _run.log_scalar("train.loss", train_loss)
        writer.add_scalar("train.loss", train_loss, epoch)

        model.eval()
        val_acc = model.test_loop(val_loader)
        _run.log_scalar("val.acc", val_acc)
        writer.add_scalar("val.acc", val_acc, epoch)

        # for baseline and baseline++, we don't use validation here so we let acc = -1
        if val_acc > max_acc:
            print("--> Best model! save...")
            max_acc = val_acc
            outfile = os.path.join(checkpoint_dir, "best_model.pth")
            torch.save(
                {
                    "epoch": epoch,
                    "state": model.state_dict(),
                    "optimizer_state": optimizer.state_dict(),
                    "max_acc": max_acc,
                },
                outfile,
            )

        if (epoch % save_freq == 0) or (epoch == stop_epoch - 1):
            outfile = os.path.join(checkpoint_dir, "last_model.pth")
            torch.save(
                {
                    "epoch": epoch,
                    "state": model.state_dict(),
                    "optimizer_state": optimizer.state_dict(),
                    "max_acc": max_acc,
                },
                outfile,
            )

        writer.flush()

    return model


@ex.automain
def main(method, start_epoch, optimization, save_freq, tag, seed, _run):
    print("using config: ", _run.config)
    print("save_dir: ", get_save_dir())

    validate_config()

    _set_seed(seed)

    max_acc = 0

    base_loader = get_base_loader()
    val_loader = get_val_loader()

    model = get_model()
    model = model.cuda()

    if optimization == "Adam":
        optimizer = torch.optim.Adam(model.parameters())
    else:
        raise ValueError("Unknown optimization, please define by yourself")

    stop_epoch = get_stop_epoch()

    if method == "maml" or method == "maml_approx":
        stop_epoch *= model.n_task  # maml use multiple tasks in one update

    writer = SummaryWriter(os.path.join(RUN_DIR, EXPERIMENT_NAME, tag, _run._id))

    model = train(
        base_loader,
        val_loader,
        model,
        optimizer,
        start_epoch,
        stop_epoch,
        os.path.join(get_save_dir(), str(_run._id)),
        writer,
        save_freq,
        max_acc,
        _run,
    )

# Test Loop

In [ ]:
import json
import tqdm
import torch
import numpy as np
import random
import torch.optim
import torch.utils.data.sampler
import os
import time

import configs
import backbone
import data.feature_loader as feat_loader
from data.datamgr import SetDataManager
from methods.baselinefinetune import BaselineFinetune
from methods.protonet import ProtoNet
from methods.ove_polya_gamma_gp import OVEPolyaGammaGP, PredictiveOVEPolyaGammaGP
from methods.logistic_softmax_gp import LogisticSoftmaxGP, PredictiveLogisticSoftmaxGP
from methods.bayesian_maml import BayesianMAML, ChaserBayesianMAML
from methods.gpnet import GPNet
from methods.matchingnet import MatchingNet
from methods.relationnet import RelationNet
from methods.maml import MAML
from io_utils import model_dict, parse_args, get_best_file, get_assigned_file

from methods.ove_polya_gamma_gp import kernel_ingredient

from calibrate import ECELoss

from sacred import Experiment
from sacred.observers import FileStorageObserver

EXPERIMENT_NAME = "test"


def get_save_dir():
    return os.path.join("runs", EXPERIMENT_NAME)


ex = Experiment(EXPERIMENT_NAME, ingredients=[kernel_ingredient])
ex.observers.append(FileStorageObserver(get_save_dir()))


@ex.capture
def get_checkpoint_dir(_run):
    return os.path.join(get_save_dir(), str(_run._id))


@ex.config
def get_config():
    # where runs are located
    run_dir = "runs/train"

    # job id to evaluate
    job_id = -1

    # saved feature from the model trained in x epoch, use the best model if x is -1
    save_iter = -1

    # number of episodes to test
    num_episodes = 600

    # relationnet_softmax replace L2 norm with softmax to expedite training,
    # maml_approx use first-order approximation in the gradient for efficiency
    # if default, match whatever setting was found in the job config
    # baseline/baseline++/protonet/matchingnet/relationnet{_softmax}/maml{_approx}
    method = "default"

    # baseline and baseline++ only use this parameter in finetuning
    # number of labeled data in each class, same as n_support
    n_shot = 5

    # baseline and baseline++ only use this parameter in finetuning
    # class num to classify for testing (validation)
    test_n_way = 5

    # default novel, but you can also test base/val class accuracy if you want
    # base/val/novel
    split = "novel"

    # further adaptation in test time or not
    adaptation = False

    # Repeat the test N times with different seeds and take the mean. The seeds range is [seed, seed+repeat]
    repeat = 5

    # number of draws for polya-gamma gps
    num_draws = None

    # number of steps for polya-gamma gps
    num_steps = None

    # Seed for Numpy and pyTorch. Default: 0 (None)
    seed = 0

    # tag (for logging purposes)
    tag = "default"

    # command allows specification of which evaluation to run
    command = "evaluate"

    # command = shot_sweep
    shot_sweep_min_shot = 1
    shot_sweep_max_shot = 20

    # None for no noise for default, otherwise 0-14
    noise_type = None

    # None for no noise, otherwise 1-5
    noise_strength = None

    run_prefix = ""


def _set_seed(seed, verbose=True):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        if verbose:
            print("[INFO] Setting SEED: " + str(seed))
    else:
        if verbose:
            print("[INFO] Setting SEED: None")


@ex.capture
def feature_evaluation(
    cl_data_file, model, test_n_way, n_shot, n_query=15, adaptation=False
):
    class_list = cl_data_file.keys()

    select_class = random.sample(class_list, test_n_way)
    z_all = []
    for cl in select_class:
        img_feat = cl_data_file[cl]
        perm_ids = np.random.permutation(len(img_feat)).tolist()
        z_all.append(
            [np.squeeze(img_feat[perm_ids[i]]) for i in range(n_shot + n_query)]
        )  # stack each batch

    z_all = torch.from_numpy(np.array(z_all))

    model.n_query = n_query
    if adaptation:
        scores = model.set_forward_adaptation(z_all, is_feature=True)
    else:
        scores = model.set_forward(z_all, is_feature=True)
    pred = scores.data.cpu().numpy().argmax(axis=1)
    y = np.repeat(range(test_n_way), n_query)
    acc = np.mean(pred == y) * 100
    return acc, {"logits": scores, "targets": y}


@ex.capture
def validate_config(job_id):
    # job id checks
    assert job_id != -1, "must specify which job id to evaluate"


@ex.capture
def get_model(test_n_way, n_shot, num_draws, num_steps):
    model = get_job_config()["model"]
    dataset = get_job_config()["dataset"]
    method = get_method()
    few_shot_params = dict(n_way=test_n_way, n_support=n_shot)

    if method == "baseline":
        return BaselineFinetune(model_dict[model], **few_shot_params)
    elif method == "baseline++":
        return BaselineFinetune(model_dict[model], loss_type="dist", **few_shot_params)
    elif method == "ove_polya_gamma_gp":
        return OVEPolyaGammaGP(model_dict[model], **few_shot_params)
    elif method == "predictive_ove_polya_gamma_gp":
        model = PredictiveOVEPolyaGammaGP(model_dict[model], **few_shot_params)
        return model
    elif method == "logistic_softmax_gp":
        return LogisticSoftmaxGP(model_dict[model], **few_shot_params)
    elif method == "predictive_logistic_softmax_gp":
        return PredictiveLogisticSoftmaxGP(model_dict[model], **few_shot_params)
    elif method == "bayesian_maml":
        return BayesianMAML(
            model_dict[model],
            num_draws=num_draws,
            num_steps=num_steps,
            **few_shot_params
        )
    elif method == "chaser_bayesian_maml":
        return ChaserBayesianMAML(
            model_dict[model],
            num_draws=num_draws,
            num_steps=num_steps,
            **few_shot_params
        )
    elif method == "gpnet":
        return GPNet(model_dict[model], **few_shot_params)
    elif method == "protonet":
        return ProtoNet(model_dict[model], **few_shot_params)
    elif method == "matchingnet":
        return MatchingNet(model_dict[model], **few_shot_params)
    elif method in ["relationnet", "relationnet_softmax"]:
        if model == "Conv4":
            feature_model = backbone.Conv4NP
        elif model == "Conv6":
            feature_model = backbone.Conv6NP
        elif model == "Conv4S":
            feature_model = backbone.Conv4SNP
        else:
            feature_model = lambda: model_dict[model](flatten=False)
        loss_type = "mse" if method == "relationnet" else "softmax"
        return RelationNet(feature_model, loss_type=loss_type, **few_shot_params)
    elif method in ["maml", "maml_approx"]:
        backbone.ConvBlock.maml = True
        backbone.SimpleBlock.maml = True
        backbone.BottleneckBlock.maml = True
        backbone.ResNet.maml = True
        model = MAML(
            model_dict[model], approx=(method == "maml_approx"), **few_shot_params
        )
        if dataset in [
            "omniglot",
            "cross_char",
        ]:  # maml use different parameter in omniglot
            model.n_task = 32
            model.task_update_num = 1
            model.train_lr = 0.1
        return model
    else:
        raise ValueError("unknown method {}".format(method))


@ex.capture
def get_job_dir(run_dir, job_id):
    return os.path.join(run_dir, str(job_id))


@ex.capture
def get_job_config(run_dir, job_id, run_prefix):
    with open(os.path.join(get_job_dir(), str(run_prefix), "config.json")) as f:
        return json.load(f)


@ex.capture
def get_checkpoint_file(save_iter):
    job_dir = get_job_dir()

    if save_iter != -1:
        return get_assigned_file(job_dir, save_iter)
    else:
        return get_best_file(job_dir)


@ex.capture
def load_model(n_shot, test_n_way, num_draws, num_steps, method):
    model = get_model(
        n_shot=n_shot, test_n_way=test_n_way, num_draws=num_draws, num_steps=num_steps
    )
    model = model.cuda()

    # for baseline/baseline++ just use feature evaluation
    if get_method() not in ["baseline", "baseline++"]:
        state_dict = torch.load(get_checkpoint_file())["state"]

        # model.num_steps = 1

        # # TODO: configure this better
        if method != "default":
            print("method is not default. Assuming transfer from baseline to gp...")
            state_dict["kernel.output_scale_raw"] = torch.Tensor([1.0]).log()

            for k in [
                "classifier.weight",
                "classifier.bias",
                "classifier.L.weight_g",
                "classifier.L.weight_v",
            ]:
                if k in state_dict:
                    del state_dict[k]

        model.load_state_dict(state_dict)

    model.eval()

    if num_draws is not None:
        model.num_draws = num_draws

    if num_steps is not None:
        model.num_steps = num_steps

    return model


@ex.capture
def get_method(method):
    if method == "default":
        return get_job_config()["method"]
    else:
        return method


def get_image_size():
    model = get_job_config()["model"]
    dataset = get_job_config()["dataset"]
    if "Conv" in model:
        if dataset in ["omniglot", "cross_char"]:
            return 28
        else:
            return 84
    else:

        return 224


@ex.capture
def get_split_file(split):
    dataset = get_job_config()["dataset"]
    if dataset == "cross":
        if split == "base":
            return configs.data_dir["miniImagenet"] + "all.json"
        else:
            return configs.data_dir["CUB"] + split + ".json"
    elif dataset == "cross_char":
        if split == "base":
            return configs.data_dir["omniglot"] + "noLatin.json"
        else:
            return configs.data_dir["emnist"] + split + ".json"
    else:
        return configs.data_dir[dataset] + split + ".json"


@ex.capture
def get_feature_file(split):
    ret = os.path.join(get_job_dir(), "{}_features.hdf5".format(split))
    if os.path.isfile(ret):
        return ret
    else:
        return None


@ex.capture
def get_loader(
    iter_num, test_n_way, n_shot, method, noise_type, noise_strength, command
):
    print("loading with {:d} way and {:d} shot".format(test_n_way, n_shot))
    feature_file = get_feature_file()

    if feature_file is not None and noise_type is None and command != "ooe":
        return feat_loader.init_loader(feature_file)
    else:
        datamgr = SetDataManager(
            get_image_size(),
            n_eposide=iter_num,
            n_query=15,
            n_way=test_n_way,
            n_support=n_shot,
        )
        if noise_type is None:
            return datamgr.get_data_loader(get_split_file(), aug=False)
        else:
            return datamgr.get_noisy_data_loader(
                get_split_file(), noise_type, noise_strength
            )


def repeat_iterator(iterable):
    while True:
        for item in iterable:
            yield item


class EpochLoader:
    def __init__(self, iterable, num_episodes):
        self.iterable = repeat_iterator(iterable)
        self.num_episodes = num_episodes

    def __len__(self):
        return self.num_episodes

    def __iter__(self):
        for _ in range(self.num_episodes):
            yield self.convert_to_episode(next(self.iterable))

    def canonicalize(self, inputs, targets):
        assert inputs.size(0) == 1
        assert targets.size(0) == 1
        inputs = inputs[0]
        targets = targets[0]

        class_counts = torch.bincount(targets)
        assert torch.all(
            class_counts.eq(class_counts[0])
        ), "classes not balanced, cannot convert"

        shot = class_counts[0].item()
        way = class_counts.size(0)

        assert (
            targets.size(0) == shot * way
        ), "number of examples does not match shot * way"

        # reshape to class batched format
        inputs = inputs.reshape(way, shot, *inputs.size()[1:])
        targets = targets.reshape(way, shot)

        way_permutation = targets[:, 0].argsort()

        inputs = inputs[way_permutation]
        targets = targets[way_permutation]

        assert torch.all(
            targets.eq(torch.arange(way).unsqueeze(-1))
        ), "problem with class permutation"

        return inputs, targets

    def convert_to_episode(self, sample):
        train_inputs, train_targets = self.canonicalize(*sample["train"])
        test_inputs, test_targets = self.canonicalize(*sample["test"])

        return (
            torch.cat([train_inputs, test_inputs], 1),
            torch.cat([train_targets, test_targets], 1),
        )


def load_feature_extractor():
    method = get_job_config()["method"]
    model = get_job_config()["model"]

    if method in ["relationnet", "relationnet_softmax"]:
        if model == "Conv4":
            extractor = backbone.Conv4NP()
        elif model == "Conv6":
            extractor = backbone.Conv6NP()
        elif model == "Conv4S":
            extractor = backbone.Conv4SNP()
        else:
            extractor = model_dict[model](flatten=False)
    elif method in ["maml", "maml_approx"]:
        raise ValueError("MAML do not support save feature")
    else:
        extractor = model_dict[model]()

    extractor = extractor.cuda()

    state = torch.load(get_checkpoint_file())["state"]
    state_keys = list(state.keys())
    for i, key in enumerate(state_keys):
        if "feature." in key:
            newkey = key.replace(
                "feature.", ""
            )  # an architecture model has attribute 'feature', load architecture feature to backbone by casting name from 'feature.trunk.xx' to 'trunk.xx'
            state[newkey] = state.pop(key)
        else:
            state.pop(key)

    extractor.load_state_dict(state)
    extractor.eval()

    return extractor


@ex.capture
def single_test(model, n_shot, test_n_way, split, adaptation, num_episodes):
    loader = get_loader(num_episodes, n_shot=n_shot, test_n_way=test_n_way)

    if adaptation:
        # We perform adaptation on MAML simply by updating more times.
        model.task_update_num = 100

    if isinstance(loader, dict):
        acc_all = []
        stats_all = []
        pbar = tqdm.tqdm(range(num_episodes))
        for _ in pbar:
            acc, stats = feature_evaluation(
                loader,
                model,
                n_shot=n_shot,
                test_n_way=test_n_way,
                adaptation=adaptation,
            )
            acc_all.append(acc)
            stats_all.append(stats)
            pbar.set_description("Acc {:f}".format(np.mean(acc_all)))
        acc_mean = np.mean(acc_all)
        acc_std = np.std(acc_all)

        stats_final = {}
        for k in stats_all[0].keys():
            stats_final[k] = (
                torch.cat([torch.as_tensor(stats[k]) for stats in stats_all], 0)
                .detach()
                .cpu()
            )

        return {"acc_mean": acc_mean, "acc_std": acc_std, "stats": stats_final}
    else:
        if get_method() in ["baseline", "baseline++"]:
            feature_extractor = load_feature_extractor()
            return model.test_loop(
                loader,
                use_progress=True,
                return_stats=True,
                feature_extractor=feature_extractor,
            )
        else:
            return model.test_loop(loader, use_progress=True, return_stats=True)


@ex.capture
def ooe_evaluation(model, n_shot, test_n_way, split, adaptation, num_episodes):
    loader = get_loader(num_episodes, n_shot=n_shot, test_n_way=2 * test_n_way)

    if adaptation:
        # We perform adaptation on MAML simply by updating more times.
        model.task_update_num = 100

    if get_method() in ["baseline", "baseline++"]:
        feature_extractor = load_feature_extractor()
    else:
        feature_extractor = None

    targets_all = []
    logits_all = []

    pbar = tqdm.tqdm(loader)
    for x, _ in pbar:
        # 2C x N x ...
        x_support = x[:test_n_way, :n_shot]
        x_query = x[:test_n_way, n_shot:]
        x_distractor = x[test_n_way:, n_shot:]

        x = torch.cat([x_support, x_query, x_distractor], 1)
        model.n_query = x.size(1) - n_shot

        if feature_extractor is not None:
            x_flat = x.view(-1, *x.size()[2:])
            x_flat = feature_extractor(x_flat.cuda())
            x = x_flat.view(*x.size()[:2], -1)

        if isinstance(model, GPNet):
            _, _, _, scores = model.correct(x)
            logits_all.append(scores)
        else:
            scores = model.set_forward(x)
            logits_all.append(scores.cpu().detach().numpy())

        y_query = np.repeat(range(model.n_way), model.n_query)
        y_query = y_query.reshape(model.n_way, -1)
        y_query[:, y_query.shape[1] // 2 :] = -1
        y_query = y_query.reshape(-1)

        targets_all.append(y_query)

    return {
        "stats": {
            "logits": torch.as_tensor(np.concatenate(logits_all, 0)),
            "targets": torch.as_tensor(np.concatenate(targets_all, 0)),
        }
    }


@ex.capture
def shot_sweep(num_episodes, shot_sweep_min_shot, shot_sweep_max_shot, num_draws, _run):
    for shot in range(shot_sweep_min_shot, shot_sweep_max_shot + 1):
        target_file = os.path.join(
            get_checkpoint_dir(), "results_shot-{:02d}.pth".format(shot)
        )
        if os.path.isfile(target_file):
            continue

        model = load_model(n_shot=shot)
        if num_draws is not None:
            model.num_draws = num_draws

        results = single_test(model, n_shot=shot, num_episodes=num_episodes)

        _run.log_scalar("shot", shot)
        _run.log_scalar("acc_mean", results["acc_mean"])
        _run.log_scalar("acc_std", results["acc_std"])
        print(
            "{:d} shot: {:4.2f} +/- {:4.2f}".format(
                shot, results["acc_mean"], results["acc_std"]
            )
        )

        torch.save(results, target_file)


@ex.capture
def way_sweep(num_episodes, shot_sweep_min_shot, shot_sweep_max_shot, num_draws, _run):
    for way in range(max(2, shot_sweep_min_shot), shot_sweep_max_shot + 1):
        target_file = os.path.join(
            get_checkpoint_dir(), "results_way-{:02d}.pth".format(way)
        )
        if os.path.isfile(target_file):
            continue

        model = load_model(test_n_way=way)
        if num_draws is not None:
            model.num_draws = num_draws

        results = single_test(model, test_n_way=way, num_episodes=num_episodes)

        _run.log_scalar("way", way)
        _run.log_scalar("acc_mean", results["acc_mean"])
        _run.log_scalar("acc_std", results["acc_std"])

        print(
            "{:d} way: {:4.2f} +/- {:4.2f}".format(
                way, results["acc_mean"], results["acc_std"]
            )
        )

        torch.save(results, target_file)


@ex.capture
def scale_sweep(num_episodes, num_draws):
    print("running scale_sweep")

    results_all = []

    max_bias = 1.5
    num_points = 11

    for exp in torch.linspace(-max_bias, max_bias, num_points + 1):
        model = load_model()
        if num_draws is not None:
            model.num_draws = num_draws
        model.kernel.output_scale_raw.data.fill_(
            model.kernel.output_scale_raw.item() + exp
        )
        print("scale = ", model.kernel.output_scale_raw[:].exp())

        results = single_test(model)
        print("{:0.2f} scale: {:4.2f}".format(exp, results["acc_mean"]))

        results_all.append((exp, results))

    return results_all


@ex.capture
def noise_sweep(num_episodes, num_draws):
    print("running noise_sweep")

    for noise in [0.0, 1e-2, 1e-1, 1e0, 1e1]:
        model = load_model()
        if num_draws is not None:
            model.num_draws = num_draws
        model.noise = noise
        print("noise = ", model.noise)
        loader = get_loader(iter_num=num_episodes)
        acc_mean = model.test_loop(loader, use_progress=True)
        print("{:f} noise: {:4.2f}".format(noise, acc_mean))


@ex.automain
def main(command, seed, repeat, _run):
    print("using config: ", _run.config)
    print("save_dir: ", get_save_dir())

    validate_config()

    if command == "evaluate":
        accuracy_list = []
        results_all = []

        # repeat the test N times changing the seed in range [seed, seed+repeat]
        for i in range(seed, seed + repeat):
            if seed != 0:
                _set_seed(i)
            else:
                _set_seed(0)

            model = load_model()
            results = single_test(model)
            results_all.append(results)
            accuracy_list.append(results["acc_mean"])
            _run.log_scalar("acc", results["acc_mean"])
        print("-----------------------------")
        print(
            "Seeds = %d | Overall Test Acc = %4.2f%% +- %4.2f%%"
            % (repeat, np.mean(accuracy_list), np.std(accuracy_list))
        )
        print("-----------------------------")
        torch.save(results_all, os.path.join(get_checkpoint_dir(), "results.pth"))

        logits = torch.cat(
            [result["stats"]["logits"] for result in results_all], 0
        ).cuda()
        targets = torch.cat(
            [result["stats"]["targets"] for result in results_all], 0
        ).cuda()

        ece_module = ECELoss().cuda()
        ece_val = ece_module.forward(logits, targets)
        print("ece: ", ece_val)
        _run.log_scalar("ece", ece_val.item())
    elif command == "shot_sweep":
        _set_seed(seed)
        shot_sweep()
    elif command == "way_sweep":
        _set_seed(seed)
        way_sweep()
    elif command == "scale_sweep":
        _set_seed(seed)
        results_all = scale_sweep()
        torch.save(results_all, os.path.join(get_checkpoint_dir(), "results.pth"))
    elif command == "noise_sweep":
        _set_seed(seed)
        noise_sweep()
    elif command == "ooe":
        results_all = []
        # repeat the test N times changing the seed in range [seed, seed+repeat]
        for i in range(seed, seed + repeat):
            if seed != 0:
                _set_seed(i)
            else:
                _set_seed(0)

            model = load_model()
            results = ooe_evaluation(model)
            results_all.append(results)
        torch.save(results_all, os.path.join(get_checkpoint_dir(), "results.pth"))
    else:
        raise ValueError("unknown command {}".format(command))

# Train on emnist

## Train Main <h1> Method=ove_polya_gamma_gp

### n shot =1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=ove_polya_gamma_gp dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
INFO - train - Running command 'main'
INFO - train - Started run with ID "8"
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epoch

### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=ove_polya_gamma_gp dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "9"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epoch

## Train Main <h1> Method=predictive_ove_polya_gamma_gp

### n shot =1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=predictive_ove_polya_gamma_gp dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


INFO - train - Running command 'main'
INFO - train - Started run with ID "13"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'predictive_ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(

### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=predictive_ove_polya_gamma_gp dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "14"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'predictive_ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(

## Train Main <h1> Method= bayesian_maml

### n shot = 1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=bayesian_maml dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


INFO - train - Running command 'main'
INFO - train - Started run with ID "10"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epochs: 1

### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=bayesian_maml dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "10"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epochs: 5

## Train Main <h1> Method= chaser_bayesian_maml

### n shot = 1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=chaser_bayesian_maml dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


INFO - train - Running command 'main'
INFO - train - Started run with ID "11"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'chaser_bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total ep

### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=chaser_bayesian_maml dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "12"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'chaser_bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total ep

# Train on MNIST

## Prepare Dataset

In [ ]:
import torchvision.datasets as datasets
mnist_dataset = datasets.MNIST(root='emnist', train=True, download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 313951922.53it/s]

Extracting content/emnist/MNIST/raw/train-images-idx3-ubyte.gz to content/emnist/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 41442249.00it/s]


Extracting content/emnist/MNIST/raw/train-labels-idx1-ubyte.gz to content/emnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 162543278.10it/s]

Extracting content/emnist/MNIST/raw/t10k-images-idx3-ubyte.gz to content/emnist/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 4192457.92it/s]

Extracting content/emnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to content/emnist/MNIST/raw



In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "sepehrabdipourasl"
os.environ['KAGGLE_KEY'] = "93a0e50c12e91c63777784b17594a75d"
!kaggle datasets download -d jidhumohan/mnist-png


 97% 57.0M/58.6M [00:04<00:00, 20.5MB/s]
100% 58.6M/58.6M [00:04<00:00, 13.8MB/s]


In [ ]:
!unzip mnist-png.zip

Streaming output truncated to the last 5000 lines.
  inflating: mnist_png/training/9/1820.png  
  inflating: mnist_png/training/9/18211.png  
  inflating: mnist_png/training/9/18228.png  
  inflating: mnist_png/training/9/18241.png  
  inflating: mnist_png/training/9/18251.png  
  inflating: mnist_png/training/9/1826.png  
  inflating: mnist_png/training/9/18277.png  
  inflating: mnist_png/training/9/18296.png  
  inflating: mnist_png/training/9/183.png  
  inflating: mnist_png/training/9/18350.png  
  inflating: mnist_png/training/9/18355.png  
  inflating: mnist_png/training/9/18374.png  
  inflating: mnist_png/training/9/18377.png  
  inflating: mnist_png/training/9/18382.png  
  inflating: mnist_png/training/9/18387.png  
  inflating: mnist_png/training/9/18401.png  
  inflating: mnist_png/training/9/18405.png  
  inflating: mnist_png/training/9/18415.png  
  inflating: mnist_png/training/9/18437.png  
  inflating: mnist_png/training/9/18441.png  
  inflating: mnist_png/training/9

In [ ]:
!python invert_emnist.py
!python write_cross_char_valnovel_filelist.py

## Train Main <h1> Method=ove_polya_gamma_gp

### n shot =1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=ove_polya_gamma_gp dataset=mnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


INFO - train - Running command 'main'
INFO - train - Started run with ID "5"
using config:  {'seed': 0, 'dataset': 'mnist', 'model': 'Conv4', 'method': 'ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epochs

### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=ove_polya_gamma_gp dataset=mnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "6"
using config:  {'seed': 0, 'dataset': 'mnist', 'model': 'Conv4', 'method': 'ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epochs

## Train Main <h1> Method=predictive_ove_polya_gamma_gp

### n shot =1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=predictive_ove_polya_gamma_gp dataset=mnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


INFO - train - Running command 'main'
INFO - train - Started run with ID "7"
using config:  {'seed': 0, 'dataset': 'mnist', 'model': 'Conv4', 'method': 'predictive_ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
T

### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=predictive_ove_polya_gamma_gp dataset=mnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "8"
using config:  {'seed': 0, 'dataset': 'mnist', 'model': 'Conv4', 'method': 'predictive_ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
T

## Train Main <h1> Method= bayesian_maml

### n shot = 1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=bayesian_maml dataset=mnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


INFO - train - Running command 'main'
INFO - train - Started run with ID "1"
using config:  {'seed': 0, 'dataset': 'mnist', 'model': 'Conv4', 'method': 'bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epochs: 50


### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=bayesian_maml dataset=mnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "2"
using config:  {'seed': 0, 'dataset': 'mnist', 'model': 'Conv4', 'method': 'bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epochs: 50


## Train Main <h1> Method= chaser_bayesian_maml

### n shot = 1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=chaser_bayesian_maml dataset=mnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


INFO - train - Running command 'main'
INFO - train - Started run with ID "3"
using config:  {'seed': 0, 'dataset': 'mnist', 'model': 'Conv4', 'method': 'chaser_bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epoc

### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=chaser_bayesian_maml dataset=mnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "4"
using config:  {'seed': 0, 'dataset': 'mnist', 'model': 'Conv4', 'method': 'chaser_bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epoc

# Train on Chinese MNIST

## Prepare Dataset

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "sepehrabdipourasl"
os.environ['KAGGLE_KEY'] = "93a0e50c12e91c63777784b17594a75d"
!kaggle datasets download -d gpreda/chinese-mnist


 30% 5.00M/16.5M [00:00<00:00, 39.7MB/s]
100% 16.5M/16.5M [00:00<00:00, 76.8MB/s]


In [ ]:
!unzip chinese-mnist.zip

Streaming output truncated to the last 5000 lines.
  inflating: emnist/5/input_100_4_6.jpg  
  inflating: emnist/5/input_100_5_6.jpg  
  inflating: emnist/5/input_100_6_6.jpg  
  inflating: emnist/5/input_100_7_6.jpg  
  inflating: emnist/5/input_100_8_6.jpg  
  inflating: emnist/5/input_100_9_6.jpg  
  inflating: emnist/5/input_10_10_6.jpg  
  inflating: emnist/5/input_10_1_6.jpg  
  inflating: emnist/5/input_10_2_6.jpg  
  inflating: emnist/5/input_10_3_6.jpg  
  inflating: emnist/5/input_10_4_6.jpg  
  inflating: emnist/5/input_10_5_6.jpg  
  inflating: emnist/5/input_10_6_6.jpg  
  inflating: emnist/5/input_10_7_6.jpg  
  inflating: emnist/5/input_10_8_6.jpg  
  inflating: emnist/5/input_10_9_6.jpg  
  inflating: emnist/5/input_11_10_6.jpg  
  inflating: emnist/5/input_11_1_6.jpg  
  inflating: emnist/5/input_11_2_6.jpg  
  inflating: emnist/5/input_11_3_6.jpg  
  inflating: emnist/5/input_11_4_6.jpg  
  inflating: emnist/5/input_11_5_6.jpg  
  inflating: emnist/5/input_11_6_6.jpg 

In [ ]:
!python ove-polya-gamma-gp/filelists/emnist/invert_emnist.py
!python ove-polya-gamma-gp/filelists/emnist/write_cross_char_valnovel_filelist.py

val -OK
novel -OK


## Train Main <h1> Method=ove_polya_gamma_gp

### n shot =1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=ove_polya_gamma_gp dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


INFO - train - Running command 'main'
INFO - train - Started run with ID "6"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epoch

### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=ove_polya_gamma_gp dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "7"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epoch

## Train Main <h1> Method=predictive_ove_polya_gamma_gp

### n shot =1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=predictive_ove_polya_gamma_gp dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


INFO - train - Running command 'main'
INFO - train - Started run with ID "8"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'predictive_ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=predictive_ove_polya_gamma_gp dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "1"
using config:  {'seed': 0, 'dataset': 'mnist', 'model': 'Conv4', 'method': 'predictive_ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
T

## Train Main <h1> Method= bayesian_maml

### n shot = 1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=bayesian_maml dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


INFO - train - Running command 'main'
INFO - train - Started run with ID "6"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epochs: 50

### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=bayesian_maml dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "7"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epochs: 50

## Train Main <h1> Method= chaser_bayesian_maml

### n shot = 1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=chaser_bayesian_maml dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


INFO - train - Running command 'main'
INFO - train - Started run with ID "8"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'chaser_bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epo

### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=chaser_bayesian_maml dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "9"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'chaser_bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epo

# Train on Hindi MNIST

## Prepare Dataset

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "sepehrabdipourasl"
os.environ['KAGGLE_KEY'] = "93a0e50c12e91c63777784b17594a75d"
!kaggle datasets download -d imbikramsaha/hindi-mnist


hindi-mnist.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import shutil

# Replace 'folder_name' with the name of the folder you want to delete
# shutil.rmtree('inv_emnist')


In [ ]:
!unzip hindi-mnist.zip

Streaming output truncated to the last 5000 lines.
  inflating: Hindi-MNIST/train/7/10299.png  
  inflating: Hindi-MNIST/train/7/10300.png  
  inflating: Hindi-MNIST/train/7/10301.png  
  inflating: Hindi-MNIST/train/7/10302.png  
  inflating: Hindi-MNIST/train/7/10303.png  
  inflating: Hindi-MNIST/train/7/10304.png  
  inflating: Hindi-MNIST/train/7/10306.png  
  inflating: Hindi-MNIST/train/7/10307.png  
  inflating: Hindi-MNIST/train/7/104017.png  
  inflating: Hindi-MNIST/train/7/104018.png  
  inflating: Hindi-MNIST/train/7/104019.png  
  inflating: Hindi-MNIST/train/7/104021.png  
  inflating: Hindi-MNIST/train/7/104023.png  
  inflating: Hindi-MNIST/train/7/104024.png  
  inflating: Hindi-MNIST/train/7/104025.png  
  inflating: Hindi-MNIST/train/7/104026.png  
  inflating: Hindi-MNIST/train/7/104027.png  
  inflating: Hindi-MNIST/train/7/104028.png  
  inflating: Hindi-MNIST/train/7/104029.png  
  inflating: Hindi-MNIST/train/7/104030.png  
  inflating: Hindi-MNIST/train/7/1040

In [ ]:
# import os
# os.rename('AbdomenCT','0')
# os.rename('BreastMRI','1')
# os.rename('CXR','2')
# os.rename('ChestCT','3')
# os.rename('Hand','4')
# os.rename('HeadCT','5')



In [ ]:
import shutil
shutil.move('Hindi-MNIST/train/','emnist')
# shutil.move('1','emnist/1/')
# shutil.move('2','emnist/2/')
# shutil.move('3','emnist/3/')
# shutil.move('4','emnist/4/')
# shutil.move('5','emnist/5/')



'emnist'

In [ ]:
!python ove-polya-gamma-gp/filelists/emnist/invert_emnist.py
!python ove-polya-gamma-gp/filelists/emnist/write_cross_char_valnovel_filelist.py

val -OK
novel -OK


## Train Main <h1> Method=ove_polya_gamma_gp

### n shot =1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=ove_polya_gamma_gp dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


INFO - train - Running command 'main'
INFO - train - Started run with ID "2"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epoch

### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=ove_polya_gamma_gp dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "3"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epoch

## Train Main <h1> Method=predictive_ove_polya_gamma_gp

### n shot =1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=predictive_ove_polya_gamma_gp dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


INFO - train - Running command 'main'
INFO - train - Started run with ID "9"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'predictive_ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=predictive_ove_polya_gamma_gp dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "10"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'predictive_ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(

## Train Main <h1> Method= bayesian_maml

### n shot = 1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=bayesian_maml dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


INFO - train - Running command 'main'
INFO - train - Started run with ID "11"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epochs: 5

### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=bayesian_maml dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "12"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epochs: 5

## Train Main <h1> Method= chaser_bayesian_maml

### n shot = 1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=chaser_bayesian_maml dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


INFO - train - Running command 'main'
INFO - train - Started run with ID "4"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'chaser_bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epo

### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=chaser_bayesian_maml dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "13"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'chaser_bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total ep

# Train on Chest X-Ray Images (Pneumonia)

## Prepare Dataset

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "sepehrabdipourasl"
os.environ['KAGGLE_KEY'] = "93a0e50c12e91c63777784b17594a75d"
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.28G/2.29G [00:31<00:00, 68.2MB/s]
100% 2.29G/2.29G [00:31<00:00, 78.9MB/s]


In [ ]:
import shutil

# Replace 'folder_name' with the name of the folder you want to delete
# shutil.rmtree('inv_emnist')
shutil.rmtree('emnist')



In [ ]:
!unzip chest-xray-pneumonia.zip

Streaming output truncated to the last 5000 lines.
  inflating: chest_xray/train/NORMAL/IM-0435-0001-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0435-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0437-0001-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0437-0001-0002.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0437-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0438-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0439-0001-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0439-0001-0002.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0439-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0440-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0441-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0442-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0444-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0445-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0446-0001.jpeg  
  inflating: chest_xray/train/NORMAL/IM-0447-0001.jpeg  
  inflating:

In [ ]:
import os
os.rename('H1','0')
os.rename('H2','1')
os.rename('H3','2')
os.rename('H5','3')
os.rename('H6','4')



In [ ]:
import shutil
shutil.move('chest_xray/train/NORMAL','emnist/0')
shutil.move('chest_xray/train/PNEUMONIA','emnist/1')

# shutil.move('1','emnist/1/')
# shutil.move('2','emnist/2/')
# shutil.move('3','emnist/3/')
# shutil.move('4','emnist/4/')
# shutil.move('5','emnist/5/')



'emnist/1'

In [ ]:
!python ove-polya-gamma-gp/filelists/emnist/invert_emnist.py
!python ove-polya-gamma-gp/filelists/emnist/write_cross_char_valnovel_filelist.py

Traceback (most recent call last):
  File "/content/ove-polya-gamma-gp/filelists/emnist/invert_emnist.py", line 27, in <module>
    image_list =  [ img for img in listdir(character_folder_path) if (isfile(join(character_folder_path,img)) and img[0] != '.')]
FileNotFoundError: [Errno 2] No such file or directory: '/content/emnist/2'
Traceback (most recent call last):
  File "/content/ove-polya-gamma-gp/filelists/emnist/write_cross_char_valnovel_filelist.py", line 23, in <module>
    classfile_list_all.append( [ join(folder_path, cf) for cf in listdir(folder_path) if (isfile(join(folder_path,cf)) and cf[0] != '.')])
FileNotFoundError: [Errno 2] No such file or directory: '/content/inv_emnist/2'


## Train Main <h1> Method=ove_polya_gamma_gp

### n shot =1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=ove_polya_gamma_gp dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


INFO - train - Running command 'main'
INFO - train - Started run with ID "11"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epoc

### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=ove_polya_gamma_gp dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "12"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epoc

## Train Main <h1> Method=predictive_ove_polya_gamma_gp

### n shot =1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=predictive_ove_polya_gamma_gp dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


INFO - train - Running command 'main'
INFO - train - Started run with ID "13"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'predictive_ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(

### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=predictive_ove_polya_gamma_gp dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "14"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'predictive_ove_polya_gamma_gp', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(

## Train Main <h1> Method= bayesian_maml

### n shot = 1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=bayesian_maml dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


INFO - train - Running command 'main'
INFO - train - Started run with ID "1"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epochs: 30

### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=bayesian_maml dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "8"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epochs: 30

## Train Main <h1> Method= chaser_bayesian_maml

### n shot = 1

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=chaser_bayesian_maml dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=1


INFO - train - Running command 'main'
INFO - train - Started run with ID "3"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'chaser_bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 1, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epo

### n shot = 5

In [ ]:
!python /content/ove-polya-gamma-gp/train.py with method=chaser_bayesian_maml dataset=emnist train_aug=True kernel.name=L2LinearKernel num_draws=20 num_steps=1 n_shot=5


INFO - train - Running command 'main'
INFO - train - Started run with ID "9"
using config:  {'seed': 0, 'dataset': 'emnist', 'model': 'Conv4', 'method': 'chaser_bayesian_maml', 'train_n_way': 5, 'test_n_way': 5, 'n_shot': 5, 'train_aug': True, 'num_classes': 200, 'save_freq': 10, 'start_epoch': 0, 'stop_epoch': -1, 'optimization': 'Adam', 'num_draws': 20, 'num_steps': 1, 'sigma': None, 'tag': 'default', 'kernel': {'name': 'L2LinearKernel', 'learn_params': True}}
save_dir:  runs/train
[INFO] Setting SEED: None
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Total epo